In [ ]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']
print(openai.api_key )
file = os.environ['FILE_PATH']
print(file)

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.elastic_vector_search import ElasticVectorSearch
from langchain.vectorstores import Chroma, FAISS
from langchain.docstore.document import Document

In [ ]:
with open(file+"sample.txt") as f:
    example = f.read()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(example)
embeddings = OpenAIEmbeddings()

In [ ]:
docsearch = FAISS.from_texts(
    texts, embeddings, metadatas=[{"source": i} for i in range(len(texts))]
) 

In [ ]:
query = "What did the president say about Justice Breyer"
docs = docsearch.similarity_search(query)
#print(docs[0].page_content)

In [15]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

In [17]:
template = """You are a chatbot having a conversation with a human.

Given the following extracted parts of a long document and a question, create a final answer.

{context}

{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input", "context"], template=template
)

memory = ConversationBufferMemory(memory_key="chat_history", input_key="human_input")
chain = load_qa_chain(
    OpenAI(temperature=0), chain_type="stuff", memory=memory, prompt=prompt
)

In [18]:
query = "What did the president say about Justice Breyer"
chain({"input_documents": docs, "human_input": query}, return_only_outputs=True)

{'output_text': " I'm sorry, I don't have any information about what the president said about Justice Breyer."}

In [19]:
print(chain.memory.buffer)

Human: What did the president say about Justice Breyer
AI:  I'm sorry, I don't have any information about what the president said about Justice Breyer.


In [20]:
query = "What did the president say about Justice Breyer"
chain({"input_documents": docs, "human_input": query}, return_only_outputs=True)

{'output_text': " I'm sorry, I don't have any information about what the president said about Justice Breyer."}

In [21]:
print(chain.memory.buffer)

Human: What did the president say about Justice Breyer
AI:  I'm sorry, I don't have any information about what the president said about Justice Breyer.
Human: What did the president say about Justice Breyer
AI:  I'm sorry, I don't have any information about what the president said about Justice Breyer.
